In [1]:
from pathlib import Path

import pandas as pd

In [2]:
dates_to_visualize = [
    "2019-02",
    "2020-02",
    "2021-02",
    "2022-02",
    "2023-02",
    # "2024-02",
]

In [3]:
available_data_files = list(Path("../data/httparchive_metrics/nel_config").glob("*.parquet"))

used_data_files = list(filter(lambda file: file.stem.split("_")[-1] in dates_to_visualize, available_data_files))
used_data_files

[WindowsPath('../data/httparchive_metrics/nel_config/failure_fraction_2019-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/failure_fraction_2020-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/failure_fraction_2021-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/failure_fraction_2022-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/failure_fraction_2023-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/include_subdomains_2019-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/include_subdomains_2020-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/include_subdomains_2021-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/include_subdomains_2022-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/include_subdomains_2023-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/max_age_2019-02.parquet'),
 WindowsPath('../dat

### Success fraction configuration over months


In [4]:
# Pick out the Failure Fraction files
ff_files = list(filter(lambda file: file.stem[:file.stem.rfind("_")] == 'include_subdomains', used_data_files))
ff_files

[WindowsPath('../data/httparchive_metrics/nel_config/include_subdomains_2019-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/include_subdomains_2020-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/include_subdomains_2021-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/include_subdomains_2022-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_config/include_subdomains_2023-02.parquet')]

### Aggregate results to visualize

In [12]:
from results.result_utils import concat_data_from_files, date_to_text_format

is_data = concat_data_from_files(ff_files)
is_data['nel_include_subdomains'] = is_data['nel_include_subdomains'].astype('category')

is_data['date'] = is_data['date'].map(date_to_text_format)

is_data

,date,nel_include_subdomains,domain_count,domain_percent
0,Feb 2019,false,18,4.864865
1,Feb 2019,true,352,95.135135
0,Feb 2020,false,108494,98.987263
1,Feb 2020,true,1110,1.012737
0,Feb 2021,false,1006788,99.869359
1,Feb 2021,true,1317,0.130641
0,Feb 2022,false,970592,99.837888
1,Feb 2022,true,1576,0.162112
0,Feb 2023,false,1965629,99.405427
1,Feb 2023,true,11757,0.594573


In [13]:
result = is_data[is_data['nel_include_subdomains'] == 'true'].copy()
result['true'] = result['domain_percent']
result[['date', 'true']]

,date,true
1,Feb 2019,95.135135
1,Feb 2020,1.012737
1,Feb 2021,0.130641
1,Feb 2022,0.162112
1,Feb 2023,0.594573
